# 설치

In [1]:
from google.colab import output
!pip install chart_studio
!pip install pyecharts==1.9.1
!pip install echarts-themes-pypkg
!pip install selenium
!pip install xmltodict
!pip install snapshot-selenium
!apt-get update
# !apt install chromium-chromedriver
output.clear()

# 임포트

In [2]:
import requests
import xmltodict
import pandas as pd
from pandas import Series
import xlrd
# from pyecharts.globals import CurrentConfig, NotebookType
# from pyecharts.render import make_snapshot
# CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB
from selenium import webdriver
from urllib.request import urlopen
from urllib.parse import quote_plus
from selenium.webdriver.common.keys import Keys
import numpy as np
from datetime import date
import copy, json
import chart_studio
chart_studio.tools.set_credentials_file(username='username', api_key='api_key')
from IPython.display import IFrame
import chart_studio.plotly as py
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objs as go
import plotly.graph_objects as go
import plotly.io as pio
from pyecharts.globals import ThemeType
from pyecharts.commons.utils import JsCode
from pyecharts.charts import Bar, Bar3D, Grid, Line, Liquid, Page, Pie, Timeline, Tab, Map
from pyecharts import options as opts

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


# 데이터 추출

In [3]:
getJ=pd.read_csv('/content/drive/MyDrive/COVID19/get.csv')
edu=pd.read_csv('/content/drive/MyDrive/COVID19/edu.csv')
kind=pd.read_csv('/content/drive/MyDrive/COVID19/kind.csv')
loc=pd.read_csv('/content/drive/MyDrive/COVID19/loc.csv')
edu2020=pd.read_csv('/content/drive/MyDrive/COVID19/2020.csv')
edu2019=pd.read_csv('/content/drive/MyDrive/COVID19/2019.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning:

Columns (15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93) have mixed types.Specify dtype option on import or set low_memory=False.

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning:

Columns (14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89) have mixed types.Specify dtype option on import or set low_memory=False.



#데이터 전처리 getJ

In [4]:
getJ=getJ.iloc[1:7][:]
name=['구분']
for i in range(2,22):
  for j in range(1,13):
    name.append('20{0:02d}'.format(i)+'{0:02d}'.format(j)+'01')
getJ.iloc[0]=name
# index 설정
getJ.index=getJ.iloc[:,0]
# 1열 삭제
getJ=getJ.drop('통계표명:',axis=1)
# 날짜 변환
getJ.iloc[0]=pd.to_datetime(getJ.iloc[0])
getJ.iloc[0] = getJ.iloc[0].apply(lambda x:x.strftime('%Y%m'))
getJ.index.name=''
# 1행 컬럼으로 바꾸기
getJ.columns=getJ.iloc[0]
#년도 행 삭제
getJ=getJ.drop('구분')

In [5]:
getJ

구분,200201,200202,200203,200204,200205,200206,200207,200208,200209,200210,200211,200212,200301,200302,200303,200304,200305,200306,200307,200308,200309,200310,200311,200312,200401,200402,200403,200404,200405,200406,200407,200408,200409,200410,200411,200412,200501,200502,200503,200504,...,201809,201810,201811,201812,201901,201902,201903,201904,201905,201906,201907,201908,201909,201910,201911,201912,202001,202002,202003,202004,202005,202006,202007,202008,202009,202010,202011,202012,202101,202102,202103,202104,202105,202106,202107,202108,202109,202110,202111,202112
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
구인,"57,616","53,675","69,419","62,963","63,476","44,865","46,734","54,038","49,025","59,333","47,459","33,429","37,781","50,099","47,431","43,112","39,132","36,921","37,874","37,341","38,054","46,918","38,417","35,088","36,630","58,392","57,576","50,101","51,886","48,003","58,838","62,665","40,945","58,060","45,495","39,526","52,233","52,294","72,082","61,113",...,"110,239","132,783","118,054","114,236","117,502","103,058","126,983","120,337","113,868","110,306","112,095","103,620","112,073","110,837","109,407","111,859","109,209","112,935","97,697","78,549","91,327","105,420","104,692","115,148","109,750","120,276","125,697","130,966","133,554","124,791","170,151","158,136","150,061","172,244","163,295","179,322","159,915","172,651","187,289","197,609"
구직,"92,397","72,701","87,694","65,362","65,938","61,891","73,195","72,274","68,214","74,496","70,808","68,326","76,072","80,276","86,629","73,132","68,394","73,887","83,317","80,454","81,176","79,823","73,036","76,630","100,296","108,881","112,870","86,450","80,746","88,270","91,449","89,057","76,909","90,908","92,849","94,258","141,776","110,742","147,865","109,747",...,"186,805","227,062","204,786","203,032","305,090","208,145","262,698","240,592","217,896","219,205","248,714","207,994","205,496","229,847","216,152","225,596","302,518","244,566","292,051","266,717","246,940","265,444","276,503","246,007","271,245","284,991","245,001","355,454","458,043","273,871","365,867","320,405","296,652","312,252","343,629","320,423","294,777","298,333","300,473","328,670"
구인배수,0.62,0.74,0.79,0.96,0.96,0.72,0.64,0.75,0.72,0.80,0.67,0.49,0.51,0.62,0.55,0.59,0.57,0.50,0.45,0.46,0.47,0.59,0.53,0.46,0.37,0.54,0.51,0.58,0.64,0.54,0.64,0.70,0.53,0.64,0.49,0.42,0.37,0.47,0.49,0.56,...,0.59,0.58,0.58,0.56,0.39,0.50,0.48,0.50,0.52,0.50,0.45,0.50,0.55,0.48,0.51,0.50,0.36,0.46,0.33,0.29,0.37,0.40,0.38,0.47,0.40,0.42,0.51,0.37,0.29,0.46,0.47,0.49,0.51,0.55,0.48,0.56,0.54,0.58,0.62,0.60
취업자수,"11,845","12,559","18,686","18,019","18,067","14,370","16,133","15,524","16,194","16,223","22,321","12,038","12,994","15,740","19,541","18,257","16,707","13,946","16,360","14,258","15,186","11,009","13,258","13,194","10,639","17,083","21,315","23,094","21,214","21,634","20,098","17,925","15,941","17,730","19,153","17,931","18,338","17,755","28,243","25,377",...,"52,009","53,698","56,837","47,332","61,668","48,615","76,831","67,359","60,065","55,513","62,550","53,895","51,702","58,370","54,908","47,806","55,405","60,175","64,645","56,940","49,683","54,409","58,910","59,530","66,022","60,737","66,511","66,581","70,222","68,174","103,896","88,549","79,396","83,733","89,439","77,907","82,691","81,934","78,426","69,017"
취업률,12.8,17.3,21.3,27.6,27.4,23.2,22.0,21.5,23.7,21.8,31.5,17.6,17.1,19.6,22.6,25.0,24.4,18.9,19.6,17.7,18.7,13.8,18.2,17.2,10.6,15.7,18.9,26.7,26.3,24.5,22.0,20.1,20.7,19.5,20.6,19.0,12.9,16.0,19.1,23.1,...,27.8,23.6,27.8,23.3,20.2,23.4,29.2,28.0,27.6,25.3,25.1,25.9,25.2,25.4,25.4,21.2,18.3,24.6,22.1,21.3,20.1,20.5,21.3,24.2,24.3,21.3,27.1,18.7,15.3,24.9,28.4,27.6,26.8,26.8,26.0,24.3,28.1,27.5,26.1,21.0


In [6]:
getJ_1=getJ.iloc[[0,1,3,4]]

In [16]:
getJ1=copy.copy(getJ_1)
getJ2002=getJ1.loc[:,'200201':'200212']
getJ2003=getJ1.loc[:,'200301':'200312']
getJ2004=getJ1.loc[:,'200401':'200412']
getJ2005=getJ1.loc[:,'200501':'200512']
getJ2006=getJ1.loc[:,'200601':'200612']
getJ2007=getJ1.loc[:,'200701':'200712']
getJ2008=getJ1.loc[:,'200801':'200812']
getJ2009=getJ1.loc[:,'200901':'200912']
getJ2010=getJ1.loc[:,'201001':'201012']
getJ2011=getJ1.loc[:,'201101':'201112']
getJ2012=getJ1.loc[:,'201201':'201212']
getJ2013=getJ1.loc[:,'201301':'201312']
getJ2014=getJ1.loc[:,'201401':'201412']
getJ2015=getJ1.loc[:,'201501':'201512']
getJ2016=getJ1.loc[:,'201601':'201612']
getJ2017=getJ1.loc[:,'201701':'201712']
getJ2018=getJ1.loc[:,'201801':'201812']
getJ2019=getJ1.loc[:,'201901':'201912']
getJ2020=getJ1.loc[:,'202001':'202012']
getJ2021=getJ1.loc[:,'202101':'202112']

In [21]:
# 전체 꺾은선 그래프 + 막대그래프
attr = getJ1.columns.tolist()
total_job_rate = Line(init_opts=opts.InitOpts( theme=ThemeType.WESTEROS
                                                              ,animation_opts=opts.AnimationOpts(animation_delay=1000
                                                                                                  , animation_easing="elasticOut")))
total_job_rate.add_xaxis(attr).set_global_opts(
    title_opts=opts.TitleOpts(
        title="취업률 추세",subtitle="커서와 스크롤을 조절해 원하는 정보를 확인해보세요!"),
        yaxis_opts=opts.AxisOpts(min_='Datamin', name = "취업률",name_location="center",type_="value",is_scale=True),
        xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=0)),
        datazoom_opts=[opts.DataZoomOpts(), opts.DataZoomOpts(type_="inside")],
        legend_opts=opts.LegendOpts(pos_left="40%",legend_icon='pin'),
        tooltip_opts=opts.TooltipOpts(trigger="axis", axis_pointer_type="cross")
        ).set_series_opts(label_opts=opts.LabelOpts(is_show=False))

total_job_rate.overlap(Line(init_opts=opts.InitOpts(theme=ThemeType.WESTEROS)).add_xaxis(attr).add_yaxis('취업률',getJ1.T['취업률']))


total_job_rate.render('total_job_rate.html')

'/content/total_job_rate.html'

In [ ]:
# 2개년 꺾은선 두개 비교
getJ1819=getJ1.loc[:,'201801':'2019:12']
getJ2021=getJ1.loc[:,'202001':'2021:12']
attr1 = getJ1819.columns.tolist()
attr2 = getJ2021.columns.tolist()
getJ89_line = (Line(
        init_opts=opts.InitOpts(theme=ThemeType.WESTEROS))
       .add_xaxis(attr)
       .add_yaxis('총지수',getJ1819.T['취업률'])
       ).set_global_opts(title_opts=opts.TitleOpts(title='PRE-COVID 2개년 취업률',pos_bottom="60%"),
                        legend_opts=opts.LegendOpts(pos_bottom="55%",pos_right="10%",legend_icon='pin'),
                         tooltip_opts=opts.TooltipOpts(trigger="axis", axis_pointer_type="cross"),
                       datazoom_opts=[opts.DataZoomOpts(), opts.DataZoomOpts(type_="inside")],
       ).set_series_opts(label_opts=opts.LabelOpts(is_show=False))

getJ01_line=(Line(
        init_opts=opts.InitOpts(theme=ThemeType.WESTEROS))
       .add_xaxis(df_kb2_house_melt['년도'].unique().astype(str).tolist())
       .add_yaxis('총지수', df_kb2_house_melt[df_kb2_house_melt['주택전세가격, 전국미분양 주택'] == '총지수(서울)[2017.11=100]']['비율(%)'].tolist())
       .add_yaxis('아파트', df_kb2_house_melt[df_kb2_house_melt['주택전세가격, 전국미분양 주택'] == '아파트(서울)[2017.11=100]']['비율(%)'].tolist())
       ).set_global_opts(title_opts=opts.TitleOpts(title='주택전세지수 변동폭(서울)',pos_top='65%'),
                        legend_opts=opts.LegendOpts(pos_top="70%",pos_right="10%",legend_icon='pin'),
                         tooltip_opts=opts.TooltipOpts(trigger="axis", axis_pointer_type="cross")
       ).set_series_opts(label_opts=opts.LabelOpts(is_show=False))

house__seoul = (Line(
        init_opts=opts.InitOpts(theme=ThemeType.WESTEROS))
       .add_xaxis(df_kb1_house_melt['년도'].unique().astype(str).tolist())
       .add_yaxis('전국 미분양주택', df_kb1_house_melt[df_kb1_house_melt['주택매매가격, 전국미분양 주택'] == '서울[호]']['비율(%)'].tolist())
       ).set_global_opts(title_opts=opts.TitleOpts(title='서울 미분양 주택 수'),
                        legend_opts=opts.LegendOpts(pos_top='5%',pos_right="10%",legend_icon='pin'),
                         tooltip_opts=opts.TooltipOpts(trigger="axis", axis_pointer_type="cross")
       ).set_series_opts(label_opts=opts.LabelOpts(is_show=False))


In [20]:
# 꺾은선 그래프로 만들었으나 + 막대그래프도 포함해서
job_info = Timeline(
    init_opts=opts.InitOpts(
        theme=ThemeType.WESTEROS, 
        height='400px',
        width='600px')
).add_schema(is_auto_play=True)

for i in range(2,22):
  attr = globals()['getJ20{0:02d}'.format(i)].columns.tolist()
  job_info.add((Line(init_opts=opts.InitOpts( theme=ThemeType.WESTEROS,
                                             animation_opts=opts.AnimationOpts(
                                                 animation_delay=1000, animation_easing="elasticOut"
                                                 ))).add_xaxis(attr
                                                               ).set_global_opts(
                                                                   title_opts=opts.TitleOpts(title='20{0:02d}년 취업률 추세'.format(i),subtitle="커서와 스크롤을 조절해 원하는 정보를 확인해보세요!"),
                                                                   yaxis_opts=opts.AxisOpts(min_='Datamin', name = "취업률",name_location="center",type_="value",is_scale=True),
                                                                   xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=0)),
                                                                   legend_opts=opts.LegendOpts(pos_left="40%",legend_icon='pin'),
                                                                   tooltip_opts=opts.TooltipOpts(trigger="axis", axis_pointer_type="cross")
                                                                   ).set_series_opts(
                                                                       label_opts=opts.LabelOpts(is_show=False))).overlap(Line(init_opts=opts.InitOpts(theme=ThemeType.WESTEROS)).add_xaxis(attr).add_yaxis('취업률',globals()['getJ20{0:02d}'.format(i)].T['취업률']))
                                                                       ,'20{0:02d}년'.format(i))

job_info.render('job_info.html')


'/content/totaljobtest.html'

In [ ]:
corona_industry_percent = Timeline(
    init_opts=opts.InitOpts(
        theme=ThemeType.WESTEROS, 
        height='400px',
        width='600px')
).add_schema(is_auto_play=True)
for s in s_total['년'].unique():
    a=s_total[s_total['년']==s].loc[:,'가전/가구':'교육/학원'].values.tolist()
    b=a[0]
    c=[[i] for i in b]
    corona_industry_percent.add((Pie(init_opts=opts.InitOpts(theme=ThemeType.WESTEROS)).add("", [list(z) for z in zip(s_total.columns[1:], c)],
                    rosetype="radius",radius=["30%", "60%"]))
    .set_series_opts(label_opts=opts.LabelOpts(is_show=True,position='top'))
    .set_global_opts(title_opts=opts.TitleOpts(title="연도별 카드소비액", subtitle="마우스를 파이에 올려 소유자수 비율을 확인해보세요!"),
                     legend_opts=opts.LegendOpts(type_ = 'scroll', pos_bottom ="60%",pos_right="-3%", orient="vertical",legend_icon ='pin'))
  ,"{}년도".format(s))
    
    corona_industry_percent.render('corona_ind_total.html')